code to read a zipfile from S3 into memory  
and then extract the zipfile from memory into a designated s3 bucket/prefix location


In [1]:
import boto3
import zipfile
import io

In [30]:
src_bucket = 'am-drones'
src_filename = 'images/blue-boxes-images.zip'
dst_bucket = 'situ-aware-poc'
dst_prefix = 'proxy_footage/ashley/images'

In [8]:
print('reading zip file')
s3_resource = boto3.resource('s3')
zip_obj = s3_resource.Object(bucket_name=src_bucket, key=src_filename)
buffer = io.BytesIO(zip_obj.get()["Body"].read())
print('done')

reading zip file
done


In [9]:
z = zipfile.ZipFile(buffer)

In [17]:
numfiles = len(z.namelist())

In [31]:
for count, filename in enumerate(z.namelist()):
    file_info = z.getinfo(filename)
    print(f'uploading file {count+1} of {numfiles}               \r', end='')
    s3_resource.meta.client.upload_fileobj(
        z.open(filename),
        Bucket=dst_bucket,
        Key=f'{dst_prefix}/{filename}'
    )

In [23]:
!aws s3 ls s3://$dst_bucket/$dst_prefix/

2022-02-21 14:14:41    7621576 20220202_124432.jpg
2022-02-21 14:14:43    7883821 20220202_124439.jpg
2022-02-21 14:14:40    6858989 20220202_124444.jpg
2022-02-21 14:14:39    6910969 20220202_124501.jpg
2022-02-21 14:14:38    6427781 20220202_124516.jpg
2022-02-21 14:14:41    7591161 20220202_124542.jpg
2022-02-21 14:14:41    6926285 20220202_124545.jpg
2022-02-21 14:14:39    7986427 20220202_124552.jpg
2022-02-21 14:14:42    6899027 20220202_124601.jpg
2022-02-21 14:14:43    6362683 20220202_124608.jpg
2022-02-21 14:14:36    6068685 20220202_124737.jpg
2022-02-21 14:14:37    5108694 20220202_124742.jpg
2022-02-21 14:14:38    5413251 20220202_124745.jpg
2022-02-21 14:14:37    6554078 20220202_124750.jpg
2022-02-21 14:14:34    5825791 20220202_124806.jpg
2022-02-21 14:14:36    5913938 20220202_124809.jpg
2022-02-21 14:14:37    5329855 20220202_124817.jpg
2022-02-21 14:14:40    6207204 20220202_124826.jpg
2022-02-21 14:14:40    6907000 20220202_135008.jpg
2022-02-21 14:14:43    7209961 